In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda : 0' if torch.cuda.is_available() else 'cpu')

# dataset
train_dataset = torchvision.datasets.FashionMNIST('data', download= True, transform= transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST('data', download= True, transform= transforms.Compose([transforms.ToTensor()]))

# loader
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size= 100)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size= 100)

In [3]:
# 합성곱 신경망 생성
class FashionCNN(nn.Module):
    def __init__(self):
        super(FashionCNN, self).__init__()
        # nn.Sequential -> 계층 형태로 가독성이 뛰어남, 데이터가 각 계층을 순차적으로 지나갈 때 사용, 여러 개의 계층을 하나의 컨테이너에 구현
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels= 1, out_channels= 32, kernel_size= 3, padding= 1),
            # in_channels : 입력 채널의 수(ex. 흑백 = 1, RGB = 3)
            # out_channels : 출력 채널의 수
            # kernel_size : 커널(필터) 크기, CNN에서 학습 대상은 필터 파라미터
            # padding : 패딩 크기

            nn.BatchNorm2d(32),
            # BatchNorm2d : 학습 과정에서 각 배치 단위별로 평균과 분산을 이용해 정규화
            nn.ReLU(),
            nn.MaxPool2d(kernel_size= 2, stride= 2)
            # 이미지 크기를 축소, 풀링 계층은 입력으로 받은 출력 데이터의 크기를 줄이거나 특정 데이터를 강조하는 용도(max, avg, min pooling)
            # kernel_size : mxn 행렬 가중치
            # stride : 스트라이드 간격
        )
        
        self.layer2 = nn.Sequential(
            nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size= 3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc1 = nn.Linear(in_features= 64*6*6, out_features= 600)
        self.drop = nn.Dropout2d(0.25)
        self.fc2 = nn.Linear(in_features= 600, out_features= 120)
        self.fc3 = nn.Linear(in_features= 120, out_features= 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.drop(out)
        out = self.fc2(out)
        out = self.fc3(out)
        
        return out

In [4]:
lr = 0.001
model = FashionCNN()
model.to(device)

criterion = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr= lr)
print(model)

FashionCNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)


In [ ]:
num_epochs = 5
count = 0
loss_list = []
iteration_list = []
acc_list = []

pred_list = []
labels_list = []

for epoch in range(num_epochs):
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        train = Variable(images.view(100, 1, 28, 28))
        labels = Variable(labels)
        
        outputs = model(train)
        loss = criterion(outputs, labels)
        opt.zero_grad()
        loss.backward()
        opt.step()
        count += 1

        if not(count % 50):
            total = 0
            correct = 0
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                labels_list.append(labels)
                test = Variable(images.view(100, 1, 28, 28))
                outputs = model(test)
                predictions = torch.max(outputs, 1)[1].to(device)
                pred_list.append(predictions)
                correct += (predictions == labels).sum()
                total += len(labels)

            accuracy = correct * 100 / total
            loss_list.append(loss.data)
            iteration_list.append(count)
            acc_list.append(accuracy)

        if not (count % 500):
            print('Iteration: {}, Loss: {}, Accuracy: {}%'.format(count, loss.data, accuracy) )


Iteration: 500, Loss: 0.619032621383667, Accuracy: 84.75833129882812%
Iteration: 1000, Loss: 0.4434753656387329, Accuracy: 86.04666900634766%
Iteration: 1500, Loss: 0.356201708316803, Accuracy: 85.875%
Iteration: 2000, Loss: 0.37042149901390076, Accuracy: 87.06999969482422%
Iteration: 2500, Loss: 0.2799746096134186, Accuracy: 87.24166870117188%
Iteration: 3000, Loss: 0.29353392124176025, Accuracy: 88.13166809082031%
